- 대회 데이터 셋 오류로 인한 데이터 전처리
  - https://dacon.io/competitions/official/235745/talkboard/403708?page=1&dtype=recent

In [1]:
import pandas as pd

In [2]:
train = pd.read_csv("./data/train (2).csv")
test = pd.read_csv("./data/test (2).csv")

train.shape, test.shape

FileNotFoundError: [Errno 2] File b'./data/train (2).csv' does not exist: b'./data/train (2).csv'

In [ ]:
train.columns

In [ ]:
train.columns = ['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수',
       '10분내버스정류장수', '단지내주차면수', '등록차량수']

test.columns = ['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수',
       '10분내버스정류장수', '단지내주차면수']

### 데이터 오류로 인한 데이터 제외
- 테스트셋에서 평가 제외되는 데이터는 'C2675'(2번 사항에 해당), 'C2335', 'C1327'(3번 사항에 해당) 3개 단지입니다.

In [ ]:
train.isna().sum()

In [ ]:
train.단지코드.unique()

In [ ]:
# 우선 train 데이터 셋 확인
train.loc[ ((train['단지코드']=='C2675') | 
           (train['단지코드']=='C2335') |
           (train['단지코드']=='C1327') ) , :]

In [ ]:
# 테스트 데이터 셋 확인
test.loc[ ((test['단지코드']=='C2675') | 
           (test['단지코드']=='C2335') |
           (test['단지코드']=='C1327') ) , :].head(3)

In [ ]:
# 테스트 데이터 셋에서 세개의 코드 데이터를 없애기
test = test.loc[ ~((test['단지코드']=='C2675') | 
           (test['단지코드']=='C2335') |
           (test['단지코드']=='C1327') ) , :]
test.head()

In [ ]:
# 확인
test.loc[ ((test['단지코드']=='C2675') | 
           (test['단지코드']=='C2335') |
           (test['단지코드']=='C1327') ) , :]

### 오류 데이터 처리
- ※ 동일한 단지에 코드가 2개로 부여된 단지 코드 (3쌍) : ['C2085', 'C1397'], ['C2431', 'C1649'], ['C1036', 'C2675']
    - (참고 사항) 주차면수는 하나의 단지임을 전제로 산정된 것이고 총세대수는 두 개 단지의 합계입니다. 다만 등록차량대수는 ['C2085', 'C1397'] 단지의 경우 동일 수치

In [ ]:
train.loc[ train['단지코드']=='C2085',  "총세대수" ] = 1339
train.loc[ train['단지코드']=='C1397',  "총세대수" ] = 1339

In [ ]:
# 단지코드를 C2085,C1397 => N2085로 변경
print( train.loc[ train['단지코드']=='C2085', : ].shape  )
print( train.loc[ train['단지코드']=='C1397', : ].shape  )

In [ ]:
# 변경 후, 처리 후, 단지코드를 N을 붙여 N2085로 변경
train.loc[ train['단지코드']=='C2085',  "단지코드" ] = 'N2085'
train.loc[ train['단지코드']=='C1397',  "단지코드" ] = 'N2085'

In [ ]:
train.loc[ train['단지코드']=='N2085', : ].shape

### 오류 코드 변경
- C2431, C1649의 총세대수를 1047로 변경
- C2431, C1649의 등록차량대수를 1214로 변경
- C2431, C1649의 단지코드를 N2431로 변경

In [ ]:
a = train.loc[ train['단지코드']=='C2431', : ]
b = train.loc[ train['단지코드']=='C1649', : ]

print(  a.shape, b.shape )
print( a['총세대수'], b['총세대수'])
print( a['등록차량수'], b['등록차량수'])

In [ ]:
train.loc[ train['단지코드']=='C2431',  "총세대수" ] = 1047
train.loc[ train['단지코드']=='C1649',  "총세대수" ] = 1047

train.loc[ train['단지코드']=='C2431',  "등록차량수" ] = 1214
train.loc[ train['단지코드']=='C1649',  "등록차량수" ] = 1214

train.loc[ train['단지코드']=='C2431',  "단지코드" ] = 'N2431'
train.loc[ train['단지코드']=='C1649',  "단지코드" ] = 'N2431'

In [ ]:
train.loc[ train['단지코드']=='N2431', : ].shape

### 오류 코드 변경
- C1036의 총세대수를 1243로 변경
- C1036의 단지코드를 N1036로 변경

In [ ]:
a = train.loc[ train['단지코드']=='C2675', : ]
b = train.loc[ train['단지코드']=='C1036', : ]
a.shape, b.shape

In [ ]:
train.loc[ train['단지코드']=='C1036',  "총세대수" ] = 1243
train.loc[ train['단지코드']=='C1036',  "단지코드" ] = 'N1036'

In [ ]:
train.loc[ train['단지코드']=='N1036', : ].shape

### 오류 3
3. 단지코드 등 기입 실수로 데이터 정제 과정에서 매칭 오류 발생  
 - (오류 내용) 단지코드 등 기입 실수로 총세대수가 주차면수에 비해 과하게 많거나 적은 경우가 발생하였고, 점검 결과 일부 데이터의 단지코드, 총세대수, 주차면수 등에서 오류가 검출되었습니다.
 - (발생 원인) 원천데이터 수집 과정에서 단지 코드 등이 잘못 기입되었고 이를 인지하지 못한 채 데이터 정제를 하여 오류가 발생하였습니다.
 - (관련 데이터) 아래와 같이 총 9개 단지에서 같은 문제가 확인되었습니다. 
※ 실수가 발생한 단지 코드 (9개 단지) : ['C2335', 'C1327', 'C1095', 'C2051', 'C1218', 'C1894', 'C2483', 'C1502', 'C1988']
 - C2335, C1327 단지는 테스트셋, 나머지는 트레인셋 입니다.

### 오류 처리
- train 데이터 셋에 오류 발생 코드를 ERR04로 변경 후, 데이터 셋을 두개로 분리

In [ ]:
train.loc[ train['단지코드']=='C1095',  "단지코드" ] = 'ERR04_1095'
train.loc[ train['단지코드']=='C2051',  "단지코드" ] = 'ERR04_2051'
train.loc[ train['단지코드']=='C1218',  "단지코드" ] = 'ERR04_1218'
train.loc[ train['단지코드']=='C1894',  "단지코드" ] = 'ERR04_1894'
train.loc[ train['단지코드']=='C2483',  "단지코드" ] = 'ERR04_2483'
train.loc[ train['단지코드']=='C1502',  "단지코드" ] = 'ERR04_1502'
train.loc[ train['단지코드']=='C1988',  "단지코드" ] = 'ERR04_1988'

In [ ]:
train.loc[ train['단지코드'].str.contains('ERR'), :].shape

In [ ]:
train.loc[ train['단지코드'].str.contains('ERR'), :]

### 데이터 오류 처리 후, csv파일을 만들기

In [ ]:
train_df = train.copy()
train_df_errno = train.loc[ ~train['단지코드'].str.contains('ERR'), :]
test_df = test.copy()

In [ ]:
train_df.to_csv("train_df.csv", index=False)
train_df_errno.to_csv("train_df_errno.csv", index=False)

test_df.to_csv("test_df.csv", index=False)